In [90]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

ut = 84
lt = 24
p = 20

d = pd.read_csv("./content/BTC_2019_2023_6h.csv")

def mfi_calc(d, p=14):
    tp = (d['high'] + d['low'] + d['close']) / 3
    rmf = tp * d['volume']
    dir = [1 if tp[i] > tp[i - 1] else (-1 if tp[i] < tp[i - 1] else 0) for i in range(1, len(tp))]
    dir.insert(0, 0)
    pmf = [0 if d == -1 else rmf[i] for i, d in enumerate(dir)]
    nmf = [0 if d == 1 else rmf[i] for i, d in enumerate(dir)]
    psum = pd.Series(pmf).rolling(window=p, min_periods=1).sum()
    nsum = pd.Series(nmf).rolling(window=p, min_periods=1).sum()
    ratio = psum / nsum
    return 100 - (100 / (1 + ratio))

d['mfi'] = mfi_calc(d, p=p)

d['f'] = d['mfi'].apply(lambda x: 1 if x > ut else (-1 if x < lt else 0))

def gen_signals(d, f_col):
    cmp = 0
    sig = pd.Series(0, index=d.index, name='sig')
    tt = pd.Series('', index=d.index, name='tt')
    for i in range(len(d)):
        if d[f_col].iloc[i] == 1:
            if cmp == 0:
                sig.iloc[i] = 1
                tt.iloc[i] = 'long'
                cmp = 1
            elif cmp == -1:
                sig.iloc[i] = 2
                tt.iloc[i] = 'long'
                cmp = 1
        elif d[f_col].iloc[i] == -1:
            if cmp == 0:
                sig.iloc[i] = -1
                tt.iloc[i] = 'short'
                cmp = -1
            elif cmp == 1:
                sig.iloc[i] = -2
                tt.iloc[i] = 'short'
                cmp = -1
        elif cmp == 1:
            sig.iloc[i] = -1
            tt.iloc[i] = 'close'
            cmp = 0
        elif cmp == -1:
            sig.iloc[i] = 1
            tt.iloc[i] = 'close'
            cmp = 0
    d['signals'] = sig
    d['trade_type'] = tt

gen_signals(d, 'f')
d.to_csv("mfi.csv")